In [114]:
## Fake News Classifier

In [73]:
import numpy as np 
import pandas as pa 
import re
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import LSTM,Embedding,Bidirectional,Flatten,Dense,GlobalAveragePooling1D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras import optimizers
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.callbacks import EarlyStopping
from nltk.tokenize import word_tokenize,TweetTokenizer
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt 
import seaborn as sn 
import warnings 
import string
warnings.filterwarnings('ignore')

In [115]:
## To-Do Text Visualization

In [116]:
## Spacy Text processing EDA

In [2]:
tokenizer = TweetTokenizer()
stopwords = stopwords.words('english')
punctuation = string.punctuation
engstopwords = set(stopwords)
APPO = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"we'll" : "we will"   ,
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"
}

In [74]:
#Clean the data
class Helper_Class:
    def __init__(self):
        print('Helper Class Called')
        self.all_unique_words = None
   
    def clean_data(self,data):
        try:
            re_email = re.compile(r'[\w.-]+@[\w.-]+')
            data= re_email.sub(r'',data)
            reg_website = re.compile(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([\w+-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
            data= reg_website.sub(r'',data)
            data = data.lower()
            words = tokenizer.tokenize(data)
            words = [APPO[word] if word in APPO else word for word in words]
            clean_data = [word for word in words if word not in punctuation]
            cleaned_data =  " ".join(clean_data)
            return cleaned_data
        except Exception:
            print('Problem in document')
            print(data)
    
    def unique_words(self,data_req):
        ##Total Unique words
        vocab = data_req['description'].values
        all_words = []

        for sent in vocab:
            words = tokenizer.tokenize(sent)
            for word in words:
                all_words.append(word)
        
        unique_word = set(all_words)
        self.all_unique_words = unique_word

        print('Total Words {}'.format(len(all_words)))
        print('Unique Words {}'.format(len(unique_word)))

        vocab_length = len(unique_word)+50
        return vocab_length

    def largest_sentence(self,data_req):
        ##Largest Sentence
        length = lambda Sentence: len(tokenizer.tokenize(Sentence))
        max_legth = max(data_req['description'],key=length)
        max_legth_sent_len = len(tokenizer.tokenize(max_legth))
        return max_legth_sent_len

    def Embedded_Padded_Sent(self,data_req,vocab_length,max_legth):
        padded = self.__embedded_sent(data_req,vocab_length,max_legth)
        return padded

    def __embedded_sent(self,data_req,vocab_length,max_length):
        #calcualte embedded sentence
        embedded_sentence = [one_hot(sent,vocab_length) for sent in data_req['description']]
        padded_sentence = self.__padded_sent(embedded_sentence,max_length)
        return padded_sentence

    def __padded_sent(self,embedded_sentence,max_length):
        #Calculate padded_sent
        return  pad_sequences(embedded_sentence,max_length,padding='post')
    

In [78]:
## ALL_MODELS
class Models:
    def __init__(self,X_data,Y_data,unique_words):
        self.__X = X_data
        self.__Y = Y_data
        self.__all__unique_words = unique_words 

    def Custom_Embedding(self,vocab_length,longest_length):
        models =Sequential()
        models.add(Embedding(vocab_length,20,input_length=longest_length))
        models.add(Flatten())
        models.add(Dense(1,activation='sigmoid'))
        models.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
        print(models.summary())
        models.fit(self.__X, self.__Y, epochs=20, verbose=0)
        loss, accuracy = models.evaluate(self.__X, self.__Y, verbose=0)
        print('Accuracy: %f' % (accuracy*100))
        return models

    def LSTM_CUSTOM_EMB(self,vocab_length,longest_length):
        models = Sequential()
        models.add(Embedding(vocab_length,20,input_length=longest_length))
        models.add(LSTM(10,activation='relu'))
        models.add(Dense(1,activation='sigmoid'))
        models.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
        print(models.summary())
        models.fit(self.__X, self.__Y, epochs=2, verbose=0)
        accuracy = models.evaluate(self.__X,self.__Y,verbose=0)
        print('Accuracy: %f' % (accuracy*100))
        return models

    def word2vec_LSTM(self,location,vocab_length,longest_length):
        word_embedding_mat = self.Word2VEc_Embeddings(location,vocab_length)
        
        #Build Model model
        models = Sequential()
        models.add(Embedding(vocab_length,100,weights=[word_embedding_mat],input_length=longest_length,
        trainable=False))
        models.add(LSTM(10,activation='relu',return_sequences=True))
        models.add(GlobalAveragePooling1D())
        models.add(Dense(1,activation='sigmoid'))

        models.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
        print(models.summary())

        models.fit(self.__X, self.__Y, epochs=20, verbose=0)
        loss, accuracy = models.evaluate(self.__X, self.__Y, verbose=0)
        print('Accuracy %f'% (accuracy*100))

        return models

    ##Keras functional API
    def HYBRID_LSTM_WORD2VEC(self,location,vocab_length,longest_length):
        ## Todo
        pass

    def Word2VEc_Embeddings(self,location,vocab_length):
        word_embedding_mat = self.__read_word_embedding(location,vocab_length)
        return word_embedding_mat

    def __read_word_embedding(self,file_location,vocab_length):
        ##Give the address to read all the word embedding matruix
        location = file_location
        word_embeddings = self.__create_embedded_matrix(location,vocab_length)
        return word_embeddings

    def __create_embedded_matrix(self,location,vocab_length):
        embedding_index = KeyedVectors.load_word2vec_format(location,binary=True)
        embedding_matrix = (np.random.rand(vocab_length,100) - 0.5)/5.0

        for i,word in enumerate(self.__all__unique_words):
            if word in embedding_index:
                embedding_vector = embedding_index.get_vector(word)
                embedding_matrix[i] = embedding_vector[:100]
        
        return embedding_matrix

In [51]:
data = pa.read_csv('fake_job_postings.csv')
data_req = data[['description','fraudulent']]
print(len(data_req))
#Value count target for checking if it's imbalanced
data_req=data_req.dropna(axis=0)
print(len(data_req))
values = dict(data_req['fraudulent'].value_counts())
print('Values {}'.format(values))

17880
17879
Values {0: 17014, 1: 865}


In [55]:
helper_obj = Helper_Class()
data_req['description']  =  data_req['description'].apply(lambda x: helper_obj.clean_data(x))
vocab_length = helper_obj.unique_words(data_req)
max_legth_sent_len = helper_obj.largest_sentence(data_req)
padded_sentence = helper_obj.Embedded_Padded_Sent(data_req,vocab_length,max_legth_sent_len)

print('Max Length sentence {}'.format(max_legth_sent_len))
print('Padded_Sentence index 1 {}'.format(padded_sentence[0]))

Helper Class Called
Total Words 3133278
Unique Words 90553
Max Length sentence 2166
Padded_Sentence index 1 [71250 44266   152 ...     0     0     0]


In [81]:
helper_obj = Helper_Class()
data_req['description']  =  data_req['description'].apply(lambda x: helper_obj.clean_data(x))
vocab_length = helper_obj.unique_words(data_req)
max_legth_sent_len = helper_obj.largest_sentence(data_req)
padded_sentence = helper_obj.Embedded_Padded_Sent(data_req,vocab_length,max_legth_sent_len)

print('Max Length sentence {}'.format(max_legth_sent_len))
print('Padded_Sentence index 1 {}'.format(padded_sentence[0]))

##Divide the data
X_train,X_test,Y_train,Y_test = train_test_split(padded_sentence,data_req['fraudulent'],test_size=0.2)

Helper Class Called
Total Words 3133278
Unique Words 90552
Max Length sentence 2166
Padded_Sentence index 1 [ 4129 31209 35427 ...     0     0     0]


In [82]:
##Build Model
unique_words = helper_obj.all_unique_words
embedd_mod = Models(X_train,Y_train,unique_words)
embedd_model=embedd_mod.Custom_Embedding(vocab_length,max_legth_sent_len)
loss,accuracy = embedd_model.evaluate(X_test,Y_test,verbose=0)
print('Test Accuracy {}'.format(accuracy))

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 2166, 20)          1812040   
_________________________________________________________________
flatten_9 (Flatten)          (None, 43320)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 43321     
Total params: 1,855,361
Trainable params: 1,855,361
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 99.993008
Test Accuracy 0.9812639951705933


In [83]:
embedd_mod = Models(padded_sentence,data_req['fraudulent'],unique_words)
location = r'D:\GoogleNews-vectors-negative300.bin'
model_word2 = embedd_mod.word2vec_LSTM(location,vocab_length,max_legth_sent_len)

In [113]:
##Prediction
predicted = embedd_model.predict_classes(np.reshape(X_test[0],newshape=(1,2166)))
print('Actual class {}'.format(Y_test.values[0]))
print('Predicted class {}'.format(predicted[0][0]))

Actual class 0
Predicted class 0
